## PARIS SUBSIDIES: DATA COLLECTION

### LIBRAIRIES IMPORT

In [1]:
import pandas as pd
import numpy as np
import requests

### FUNCTIONS

In [2]:
def get_data_from_sirets(lst_sirets, lst_fields):
    """
    Retrieves information from https://public.opendatasoft.com/explore/dataset/economicref-france-sirene-v3/api/
    using a list of SIRETS as input for the query.

    Args:
        lst_sirets = list of SIRETS to query in the API.

    Returns:
        df = DataFrame with information on queried SIRETS found by the API.
        rejects = List of SIRETs not found by the API. 
    """
    # List to set to eliminate duplicates
    lst_sirets = set(lst_sirets) # In case of duplicates
    nb_input_sirets = len(lst_sirets)

    # API call setup
    base_url = 'https://public.opendatasoft.com/api/records/1.0/search/'
    params = {
        'dataset':'economicref-france-sirene-v3',
        'q':'siret='+' OR siret='.join(lst_sirets),
        'rows':nb_input_sirets
        }
    
    # API call
    r = requests.get(base_url, params=params)
    if r.status_code == 200:
        print('Request OK')
        # Save request's data feedback (JSON format) into DataFrame
        df = pd.json_normalize(r.json(), record_path=['records'])
        # Drop non-requested columns
        df.drop(columns=[col for col in df.columns if col not in lst_fields], inplace=True)
        nb_records = df.shape[0]
        print(f'Found {nb_records}/{nb_input_sirets} SIRETs')
    else:
        print(f'Error {r.status_code}')
        # Empty DataFrame is request has failed
        df = pd.DataFrame()

    # List of SIRETs not found by the API
    rejects = [s for s in lst_sirets if s not in df['fields.siret'].to_list()]

    return df, rejects



In [3]:
def nb_lines_amount(subset, numeric_field, data):
    print(f'{subset.shape[0]} rows identified.')
    print(f'''Amount at stake = {subset[numeric_field].sum()} i.e. {subset[numeric_field].sum()/data[numeric_field].sum():.2%} of total.''')

### DATA IMPORT: SUBSIDIES FILE

In [4]:
# Importing all fields as string
data = pd.read_csv('../00_DataFiles/00_Raw/subventions-accordees-et-refusees.csv', sep = ';', dtype = str)

In [5]:
# Check shape, column names and missing values
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80084 entries, 0 to 80083
Data columns (total 10 columns):
 #   Column                                           Non-Null Count  Dtype 
---  ------                                           --------------  ----- 
 0   Numéro de dossier                                80084 non-null  object
 1   Année budgétaire                                 80084 non-null  object
 2   Collectivité                                     80084 non-null  object
 3   Nom Bénéficiaire                                 80063 non-null  object
 4   Numéro Siret                                     79892 non-null  object
 5   Objet du dossier                                 80084 non-null  object
 6   Montant voté                                     80084 non-null  object
 7   Direction                                        80084 non-null  object
 8   Nature de la subvention                          80084 non-null  object
 9   Secteurs d'activités définies par l'ass

In [6]:
# Save nb of rows
nb_rows_original = data.shape[0]

In [7]:
# First 5 rows
data.head()

,Numéro de dossier,Année budgétaire,Collectivité,Nom Bénéficiaire,Numéro Siret,Objet du dossier,Montant voté,Direction,Nature de la subvention,Secteurs d'activités définies par l'association
0,2020_07586,2020,Ville de Paris,13 POUR TOUS,48905531900029,Co-financement DDCT d'un poste adultes-relais ...,4700,DDCT,Fonctionnement,Loisirs;Social;Vie et animation locale
1,2020_08053,2020,Ville de Paris,13 POUR TOUS,48905531900029,CS 2020/DASES SEPLEX,209692,DASES,Fonctionnement,Loisirs;Social;Vie et animation locale
2,2020_04885,2020,Ville de Paris,3 S : SEJOUR SPORTIF SOLIDAIRE,81147565600013,Ressourcerie Ephémère,0,DAE,Non précisée,Environnement & écologie;Sport;Vie et animatio...
3,2020_05032,2020,Ville de Paris,4 A 4 DIX HUIT,48909505900015,De Mômes et d'opérette : Des grottes de Lascau...,0,DASCO,Non précisée,Culture & Arts;Social;Sport
4,2020_02992,2020,Ville de Paris,ABC PUERICULTURE,34482479200032,Fonctionnement 2020 cité des ours 14 cité fala...,217134,DFPE,Fonctionnement,Education & formation;Santé;Social


In [8]:
# Last 5 rows
data.tail()

,Numéro de dossier,Année budgétaire,Collectivité,Nom Bénéficiaire,Numéro Siret,Objet du dossier,Montant voté,Direction,Nature de la subvention,Secteurs d'activités définies par l'association
80079,2021_03464,2021,Ville de Paris,VUE D'ENSEMBLE,53920045100018,Subvention au titre de la jeunesse,4000,DJS,Projet,Culture & Arts;Social;Vie et animation locale
80080,2021_09242,2021,Ville de Paris,WE LOVE GREEN,51069095100049,"AGR URBAINE, ALIMENTATION DURABLE, CIRCUITS CO...",0,DAE,Non précisée,Culture & Arts;Environnement & écologie;Techni...
80081,2021_01824,2021,Ville de Paris,YOUTH ID (YID),84392438200016,Dynamisons nos Quartiers,0,DDCT,Non précisée,Défense des droits et des intérêts;Education &...
80082,2021_01394,2021,Ville de Paris,ZARTS PROD,51792060900024,Websérie (tournage et diffusion),5200,DDCT,Projet,Culture & Arts;Vie et animation locale
80083,2021_06947,2021,Ville de Paris,ZUP DE CO,48899958200015,Tutorat solidaire dans 3 collèges du 18e,0,DASCO,Non précisée,Education & formation;Social;Vie et animation ...


In [9]:
# Convert 'Année budgétaire' and 'Montant voté' to integers
data['Année budgétaire'] = data['Année budgétaire'].astype(int)
data['Montant voté'] = data['Montant voté'].astype(int)

### MISSING VALUES

In [10]:
# By column
data.isna().sum()

Numéro de dossier                                    0
Année budgétaire                                     0
Collectivité                                         0
Nom Bénéficiaire                                    21
Numéro Siret                                       192
Objet du dossier                                     0
Montant voté                                         0
Direction                                            0
Nature de la subvention                              0
Secteurs d'activités définies par l'association     46
dtype: int64

In [11]:
# Zoom on lines with missing ID-type columns: 'Numéro Siret' and 'Nom Bénéficiaire'
mask_missing_id = (data['Numéro Siret'].isna()) & (data['Nom Bénéficiaire'].isna())
df_missing_id = data[mask_missing_id]
nb_lines_amount(df_missing_id, 'Montant voté', data)

21 rows identified.
Amount at stake = 238400 i.e. 0.01% of total.


In [12]:
# Lines with missing 'Nom bénéficiaire' also miss 'Numéro Siret'
# Any significant amount for which we should get a closer look at? (for subsequent manual correction)

amount_missing_id_threshold = 5000

for i in df_missing_id.index:
    if data.loc[i, "Montant voté"] >= amount_missing_id_threshold:
        print(f'Index = {i}, Amount = {data.loc[i, "Montant voté"]}, Objet = {data.loc[i, "Objet du dossier"]}')

Index = 12714, Amount = 150000, Objet = création d'un FAM pour personnes vieillissantes en situation de handicap mental
Index = 22327, Amount = 5000, Objet = Fonds du maire du 16ème arrondissement
Index = 27318, Amount = 60000, Objet = équipement et mobilier pour l'ouverture d'un foyer d'hébergement
Index = 53494, Amount = 6000, Objet = FONDS DU MAIRE DU 16ème


In [13]:
# Not much to extract: we can drop all these lines as we won't be able to process them in our analysis
# However we set up a dedicated DataFrame to store the dropped records
df_drops = pd.DataFrame()
df_drops = pd.concat([df_drops,data[mask_missing_id]], ignore_index=True)
data.drop(data[mask_missing_id].index, inplace = True)

In [14]:
# Remaining missing values
mask_missing = data['Numéro Siret'].isna()
df_missing = data[mask_missing]
nb_lines_amount(df_missing, 'Montant voté', data)
df_missing.isna().sum()

171 rows identified.
Amount at stake = 751879 i.e. 0.03% of total.


Numéro de dossier                                    0
Année budgétaire                                     0
Collectivité                                         0
Nom Bénéficiaire                                     0
Numéro Siret                                       171
Objet du dossier                                     0
Montant voté                                         0
Direction                                            0
Nature de la subvention                              0
Secteurs d'activités définies par l'association      0
dtype: int64

In [15]:
# No more missing 'Nom Bénéficiaire'
# List of all lines with a 'Nom Bénéficiaire' but without a 'Numéro Siret'
df_missing['Nom Bénéficiaire'].unique()

array(['ACTE  -  ASSOCIATION CULTURES TERRITOIRES', 'ADJ 11',
       'CENTRE MEDEM-ARBETER RING - DOUBLON',
       'COMITE DEPARTEMENTAL PARISIEN DES ECHECS (CDPE) - DOUBLON',
       'CHOEUR DU TEMPLE DE PORT-ROYAL', 'ATELIER CHAUSSURES/SOULIER',
       'BAREFOOT PRODUCTIONS', "BOL D'OR", 'CHOREDANZAS',
       'SYNDICAT DES ORGANISATEURS CULTURELS LIBRES ET ENGAGES', 'ESPERO',
       'PARIS CAPITALE DU MONDE',
       "SORBONNE POUR L'ORGANISATION DES NATIONS UNIES (SONU)",
       'ASSOCIATION LA GRANDE COCO', 'BIENNALE DE PARIS', "L'ALLUME-FEU",
       'LES CHUTES LIBRES',
       '"ACP  LA MANUFACTURE CHANSON"(DOUBLON AVEC LE 181331 CHANGEMENT DE NATURE JURIDIQUE)',
       "ASSOCIATION FRANCOPHONE DE SIMULATION EN ANESTHESIE REANIMATION ET EN MEDECINE D'URGENCE",
       "CENTRE D'ENTRAIDE PARISIEN DE LA PRESSE ET DE L'EDITION CEPPE",
       'LE POMMIER DES AMANDIERS',
       'FEDERATION INTERNATIONALE MUSIQUE ESPERANCE',
       "REGARD'EN FRANCE CRTH", 'GALAPIAT CIRQUE', 'ARTICHAUT',
 

In [16]:
# Manual input of SIRETs using https://www.sirene.fr/sirene/public/recherche

dct_nom_siret_missing = {
    'ACTE  -  ASSOCIATION CULTURES TERRITOIRES':'49836565900016',
    'ADJ 11':'85085802800010',
    'CENTRE MEDEM-ARBETER RING - DOUBLON':'77567872500018',
    'COMITE DEPARTEMENTAL PARISIEN DES ECHECS (CDPE) - DOUBLON':'42954380400042',
    'CHOEUR DU TEMPLE DE PORT-ROYAL':'84198004800018',
    'ATELIER CHAUSSURES/SOULIER':'83846023600022',
    'BAREFOOT PRODUCTIONS':'89124797500016',
    "BOL D'OR":'50965327500016',
    'CHOREDANZAS':'83136474000025',
    'ESPERO':'82375485800012',
    'PARIS CAPITALE DU MONDE':'84209696800016',
    'OISEAUX MIGRANTS':'75170501300025',
    "SORBONNE POUR L'ORGANISATION DES NATIONS UNIES (SONU)":'53964455900020',
    'ASSOCIATION LA GRANDE COCO':'84116387600029',
    'BIENNALE DE PARIS':'53783850000024',
    "L'ALLUME-FEU":'83962853400027',
    'LES CHUTES LIBRES':'84363377700019',
    'FEDERATION INTERNATIONALE MUSIQUE ESPERANCE':' 44262410200038',
    'ARTICHAUT':'83983061900012',
    'MAFALDA':'78900949500019', 
    "ARCHI M'AIDE":'79230324000016',
    'UN TEMPS POUR TOUS':'81322528100019',
    "SYNDICAT NATIONAL DE L'EDITION (RÉFÉRENCEMENT ANNULÉ ERREUR DE NATURE JURIDIQUE)":'78428506600030',
    "ARIF PARCOURS: ASSOCIATION POUR LE RECRUTEMENT, L'INSERTION ET LA FORMATION PROFESSIONNELLE":'48990092800024',
    'LES AMIS DU CENTRE KOURITA ST PHILIPPE':'49751436400023',
    "ARCHI'M'AIDE":'79230324000016',
    'ASSOCIATION PRESSE OFFSET':'84070581800010',
    'ASSOCIATION THANKS FOR NOTHING':'83118677000010',
    'CIE THEATRALE CHOUFF':'49292142400023',
    'COREDIA COMITE RENOVATION DE DIAWARA':'49021335200019',
    'ASSOCIATION FRANCO-KABYLE CULTURE ET ORIGINE - AFKCO':'84033609300017',
    'COMPAGNIE THEATRUM MUNDI':'83517873200011',
    'ESPOIR BERBERE':'83834612000010',
    'BUREAU DES SPORTS ECE':'83927330700018',
    'ARTCELERATOR':'84246834000018',
    "EPY CENTRE: CENTRE D'ÉTUDES ET DE PRATIQUES DU YOGA":'84092745300018',
    'LE LIVRE A 2 MAINS':'83507769400023',
    'GEDAN KARATE SHOTOKAN (GKS)':'83101787600028',
    'CLIC IMAGES':'82213848300026',
    "ENERCIT'IF":'83267821300014',
    'COMPAGNIE UNI VERS':'84283296600027',
    'NOGOZON PARIS':'82360070500037',
    "COMPAGNIE L'AIR DU VERSEAU":'40342904600028',
    'LA COMPAGNIE DE LA DERNIERE FIN':'84781587500012',
    'LES LIONS DE SAINT PAUL':'31162261700034',
    "LIRE C'EST LIBRE":'82873350100013',
    'LA JOIE IMMENSE (L.J.I)':'82797222500011',
    'PEACE AND LOVE DES FEMMES DU 17ÈME ARRONDISSEMENT':'79755853300015',
    'LES TALONS CASSES':'83162584300014',
    'LUTECE SPORT':'84973834900018',
    "PONTHIEU D'ABORD(S) (PDA)":'83390451900018',
    'TREFLE - ASSOCIATION DE MEDIATION ET DE CONSEIL CONJUGAL ET FAMILIAL 75 (TREFLE-AMCCF-75)':'53999614000023',
    'SOONINKARA XILLEN DO NAAME':'82809438300019',
    'ASSOCIATION FÊTE DES CARACTÈRES CHINOIS  (A.F.C.C)':'84405532700010',
    'NAVARRE SANS PEUR, ASSOCIATION AMICALE DES ANCIENS DU 5EME R.I. ET DU 5EME B.I':'49046393200023',
    'ABEILLE ET JOIE':'84324400500016',
    'QUARTIERLUD':'84042956700018',
    'EQUIPE SAINT VINCENT ALPHA 16 PARIS':'45250850000025',
    'MILO':'84531417800011',
    'AQUA 19':'41147569200017',
    'COLLECTIF AMICAL PHARMACEUTIQUE DES INTERNES AGEISTES DE LUTECE (CAPITAL)':'85114307300019',
    'IMHOTEP SANTÉ':'84412724100017',
    "ASSOCIATION SPORTIVE DU LYCEE POLYVALENT D'ETAT JB SAY":'50783550200018',
    'HANDICAP FRANCE SOUDAN DARFOUR':'83443587700016',
    'HANDINAMIK':'82330784800015',
    'ASSOCIATION POUR LA FONDATION CRESUS D INITIATIVE ECONOMIQUE ET SOCIALES - DOUBLON':'51131797600021',
    'KAYUMANGGI ASSOCIATION PHILIPPINS TRAVAILLEURS DE FRANCE':'83103722100015',
    'COMITE DEPARTEMENTAL DE CYCLOTOURISME DE PARIS CODEP 75 -DOUBLON':'80137007300018',
    'AU BAGAGE DU CANAL - DOUBLON':'83169480700015',
    'CARRE RIVE GAUCHE LE MEILLEUR DE L ART ASSOCIATION DES ANTIQUAIRES ET GALERIES D ART - DOUBLON':'82967587500019',
    'A.N.P.G.I.G. DOUBLON':'53916864100014',
    "L'AUTRE LIVRE ASSOCIATION INTERNATIONALE D'EDITEURS INDEPENDANTS DOUBLON":'49145578800035',
    "ASSOCIATION DE SANTE MENTALE ET DE LUTTE CONTRE L'ALCOOLISME DANS LE XIIIEME ARRONDISSEMENT":'77568194300053',
    "L'ÎLOT LÉON - DOUBLON":'84536854700012',
    'PARIS SAINT-GERMAIN HANDBALL ASSOCIATION - DOUBLON':'48360175300021',
    'ASSO ZITA':'80438620900030',
    'LES MOUETTES DES ECLUSES  DOUBLON':'38922339700032',
    'EDITIONS SANS CRISPATION - ASSOCIATION':'84032272100019',
    'DOUBLON - ASSOCIATION DE MEDIATION INTERCULTURELLE - AMI':'44158210300020',
    'VAYNAH':'85211596300013',
    'VELO CLUB DES VETERANS PARISIENS DOUBLON':'45274952600029',
    "DOUBLON 1 - CLUB 608 D'ECHECS DE PARIS SAINT- LOUIS":'51227777300028',
    'QUARTIER DE SOLEIL':'48467532700023',
    'INUKSUK':'40241100300021',
    'MOUVEMENT INTERNATIONAL CULTUREL DE LA JEUNESSE (MICJ)':'50221615300010',
    "CHOEUR DE CHAMBRE DE L'ILE DE FRANCE":'43124145400039',
    'JOSIANE SALONE':'83118373600014',
    'PARIS ELITE FUTSAL':'84532448200015',
    'SYNDICAT DES ORGANISATEURS CULTURELS LIBRES ET ENGAGES':'85052089100019',
    '"ACP  LA MANUFACTURE CHANSON"(DOUBLON AVEC LE 181331 CHANGEMENT DE NATURE JURIDIQUE)':'32858259800046',
    "ASSOCIATION FRANCOPHONE DE SIMULATION EN ANESTHESIE REANIMATION ET EN MEDECINE D'URGENCE":'78901249900016',
    "CENTRE D'ENTRAIDE PARISIEN DE LA PRESSE ET DE L'EDITION CEPPE":'77567873300061',
    "REGARD'EN FRANCE CRTH":'39326214200051',
    'LE POMMIER DES AMANDIERS':'79255447900016',
    'GALAPIAT CIRQUE':'50103888900017',
    '6M3 / 6MCUBE':'84779810500015',
    'LA COMMUNAUTE ECOTABLE':'84363427000014',
    'ECOLE NUMÉRIQUE SOLIDAIRE DES QUARTIERS':'85074609000016',
    "COLLECTIF L'ART AU QUOTIDIEN":'45047976100030',
    'ASSOCIATION DES PROFESSIONNELS DE SANTE DE LA CASERNE DE REUILLY':'84979055500015',
    'SOCIETE MATERNELLE LA NOUVELLE ETOILE DES ENFANTS DE FRANCE':'77566322200153',
    "BIENNALE DE L'IMAGE TANGIBLE":'83829555800012',
    'ASSOCIATION CULTURELLE ET EVENEMENTIELLE DU 15EME ARRONDISSEMENT DE PARIS (ACE 15)':'48469647100024',
    'AMICALE RETRAITES MARIE DE PARIS':'48209742500037',
    'ASSOCIATION SPORTIVE DU BON CONSEIL ASBC':'78430883500015',
    'ABAJAD':'89170173200010',
    'LES AMI-E-S DES 400 COOP':'82930357700011',
    'ASSOCIATION POUR L\'EGALITE  "APEL-EGALITE"':'50000968300029',
    'GENERATION PLUS: TON EMPLOI':'84249746300021',
    'P.P.DREAM':'40532511900023',
    'LA MAISON DES ILLUSTRATEURS':'43945385300020'
    }

In [17]:
# Fill the DataFrame with correct SIRET or drop the line if no manual SIRET

for nom in df_missing['Nom Bénéficiaire'].unique():
    try:
        # dct_nom_siret_missing[nom] returns an error if the 'nom' key does not exist in the dictionary
        data.loc[data['Nom Bénéficiaire'] == nom, 'Numéro Siret'] = dct_nom_siret_missing[nom]
    except:
        # In case of error, save problematic line in df_drops and drop it from main DataFrame
        df_drops = pd.concat([df_drops,data.loc[data['Nom Bénéficiaire'] == nom]], ignore_index=True)
        data.drop(data.loc[data['Nom Bénéficiaire'] == nom].index, axis = 0, inplace=True)

### DATA IMPORT: API CALL USING SIRETs

In [18]:
# List of unique SIRETS
sirets = data['Numéro Siret'].astype(str).map(lambda x: x.replace('.0','')).unique().tolist()

In [19]:
# API call
# https://public.opendatasoft.com/explore/dataset/economicref-france-sirene-v3/information/

# Define request_size and compute nb_requests to split the overall request in several calls
nb_rows = len(sirets)
request_size = 1000
nb_requests = int(np.ceil(nb_rows/request_size))

# DataFrame to store the results
data_sirets = pd.DataFrame()

# List to store the rejected SIRETs from the API call
sirets_rejects = list()

# List of fields to get
# Definitions: https://www.sirene.fr/sirene/public/static/liste-variables
lst_fields = [
    'fields.categorieentreprise',
    'fields.etatadministratifunitelegale',
    'fields.geolocetablissement',
    'fields.trancheeffectifsunitelegale',
    'fields.trancheeffectifsunitelegaletriable',
    'fields.siretsiegeunitelegale',
    'fields.siret',
    'fields.etablissementsiege',
    'fields.statutdiffusionunitelegale',
    'fields.sectionunitelegale',
    'fields.codepostaletablissement',
    'fields.caractereemployeurunitelegale',
    'fields.datecreationunitelegale',
    'fields.economiesocialesolidaireunitelegale',
    'fields.datecreationetablissement',
    'fields.denominationunitelegale',
    'geometry.type',
    'geometry.coordinates'
]

for i in range(nb_requests):
    print(f'Request {i+1}/{nb_requests}...')
    # Select SIRETs (slice list)
    lst_sirets = sirets[i*request_size:(i+1)*request_size]
    # API call
    df, rejects = get_data_from_sirets(lst_sirets, lst_fields)
    # Append results to data_sirets
    if not df.empty:
        print(f'Appending to dataframe...')
        data_sirets = pd.concat([data_sirets,df], ignore_index=True)
    # Store rejected SIRETs
    sirets_rejects.extend(rejects)

print(f'Number of retrieved Sirets: {data_sirets.shape[0]} ({data_sirets.shape[0]/nb_rows:.1%} of total)')
print(f'Number of rejected Sirets: {len(sirets_rejects)} ({len(sirets_rejects)/nb_rows:.1%} of total)')

Request 1/12...
Request OK
Found 997/1000 SIRETs
Appending to dataframe...
Request 2/12...
Request OK
Found 990/1000 SIRETs
Appending to dataframe...
Request 3/12...
Request OK
Found 977/1000 SIRETs
Appending to dataframe...
Request 4/12...
Request OK
Found 972/1000 SIRETs
Appending to dataframe...
Request 5/12...
Request OK
Found 974/1000 SIRETs
Appending to dataframe...
Request 6/12...
Request OK
Found 967/1000 SIRETs
Appending to dataframe...
Request 7/12...
Request OK
Found 971/1000 SIRETs
Appending to dataframe...
Request 8/12...
Request OK
Found 984/1000 SIRETs
Appending to dataframe...
Request 9/12...
Request OK
Found 976/1000 SIRETs
Appending to dataframe...
Request 10/12...
Request OK
Found 997/1000 SIRETs
Appending to dataframe...
Request 11/12...
Request OK
Found 1000/1000 SIRETs
Appending to dataframe...
Request 12/12...
Request OK
Found 187/192 SIRETs
Appending to dataframe...
Number of retrieved Sirets: 10992 (98.2% of total)
Number of rejected Sirets: 201 (1.8% of total)

In [20]:
# Let's examine the rejected SIRETs
mask_incorrect = data['Numéro Siret'].isin(sirets_rejects)
df_incorrect = data[mask_incorrect]

In [21]:
# Create dictionary to store rejected SIRETs (keys) and related 'Nom Bénéficiaire' (values)

dct_siret_nom_incorrect = dict()

for s in sirets_rejects:
    try:
        dct_siret_nom_incorrect[s] = data.loc[data.loc[data['Numéro Siret'] == s].index.min(), 'Nom Bénéficiaire']
    except:
        pass

dct_siret_nom_incorrect

{'84316531600010': 'CTEEN FRANCE',
 '50189845800018': 'CERAF SOLIDARITE',
 '789456123': 'ASSOCIATION MAISON BAKHITA - AMB',
 '90092908400011': 'LA PROTECTION SOCIALE DE VAUGIRARD - JEAN CHERIOUX',
 '51761699400010': 'DURAPOLE',
 '39819156900000': 'UNIS CITE',
 '42405599400000': 'DOCOMOMO FRANCE',
 '78976758500015': 'SCIENTIPÔLE CROISSANCE',
 '40920971500023': '"SOTTO VOCE"',
 '38515263000029': "ASSOCIATION POUR LA COORDINATION D'ACTIONS UTILES AUX PAYS EN DÉVELOPPEMENT",
 '48451470000014': 'ASSOCIATION SPORTIVE DE PASSY FULL CONTACT',
 '82528676600018': 'FOULQUES PARISIENNES',
 '82090845700014': 'INNOVATION FACTORY',
 '38921580700005': 'COMPAGNIE LA TRAVERSE',
 '823189931000.2': 'HOPHOPFOOD',
 '50391007700000': 'CHANTIERS 21',
 '44875887000027': 'VISEMPLOI',
 '42520720200014': 'POINTLIGNEPLAN',
 '750473647': 'ASSOCIATION SPORTIVE DU COLLEGE BEAUMARCHAIS',
 '50439846200000': 'LA MÉMOIRE NÉCROPOLITAINE',
 '429484876': 'ASSOCIATION SPORTIVE DU LYCÉE CONDORCET',
 '529066730 0019': "LES OIS

In [22]:
# Manual correction of SIRETs using https://www.sirene.fr/sirene/public/recherche
# and Google searches (in particular for 'Mentions légales' if the entity has a website)

dct_nom_siret_incorrect = {
    'CTEEN FRANCE':'84317531600010',
    'ASSOCIATION MAISON BAKHITA - AMB':'87995654800017',
    'CERAF SOLIDARITE':'50189854800026',
    'INNOVATION FACTORY':'79856979400034',
    'LA PROTECTION SOCIALE DE VAUGIRARD - JEAN CHERIOUX':'77568401200138',
    'ASSOCIATION POUR LA COORDINATION D\'ACTIONS UTILES AUX PAYS EN DÉVELOPPEMENT':'38415296300037',
    '"SOTTO VOCE"':'40920971500021',
    'UNIS CITE':'39819156900209',
    'ASSOCIATION SPORTIVE DE PASSY FULL CONTACT':'48451474000014',
    'DURAPOLE':'51761694200028',
    'SCIENTIPÔLE CROISSANCE':'50018581400018',
    'FOULQUES PARISIENNES':'83528676600018',
    'DOCOMOMO FRANCE':'42405599400026',
    'LES MOUETTES DES ECLUSES':'38922339700032',
    'PARI(S) JEUNE PUBLIC':'42393165800011',
    'POINTLIGNEPLAN':'45220720200014',
    'DROITS D\'URGENCE (DISSOUTE POUR FUSION/NOUVEAU COMPTE SIMPA À VENIR)':'45101859200027',
    'COMPAGNIE LA TRAVERSE':'40850767100019',
    'LES OISEAUX R\'ARTS':'52906673000019',
    'ARTISTEP\'S & TNP':'81785016700021',
    'ASSOCIATION SPORTIVE DU COLLEGE BEAUMARCHAIS':'75047364700014',
    'LA MÉMOIRE NÉCROPOLITAINE':'50439846200011',
    'LES PAS SAGES':'81434492500015',
    'ESPERANCE SPORTIVE PARISIENNE':'34396747700021',
    'FEDERATION DU CONSEIL DES MARIANNES DE LA REPUBLIQUE ET D\'EUROPE-FEDERATION CMR EUROPE':'50742967800034',
    'LES AMIS DE PÉNÉLOPE':'30446404300039',
    'VISEMPLOI':'50176049000018',
    'ZEN ART':'51217174500015',
    'ASSOCIATION PERMIS POUR REUSSIR (APPR)':'53120470900012',
    'ASSOCIATION SPORTIVE DU LYCÉE CONDORCET':'42948487600025',
    'CHANTIERS 21':'50391007700026',
    'CINÉMA(S) D\'IRAN':'79740268200026',
    'GABY SOURIRE':'47978923200010',
    'HOPHOPFOOD':'82318993100012',
    'LA GALERIE DU HAUT-PAVÉ':'32363567200013',
    'ARTISTES A LA BASTILLE':'38905172300028',
    'ASSOCIATION D\'ACCOMPAGNEMENT DES COOPERATIVES D\'HABITANTS AP HABICOOP':'48859158700024',
    'AZRIR':'50056472900027',
    'ASSOCIATION HISTORIQUE DU LYCÉE JULES-FERRY (PARIS 9E ARRDT.)':'79740332600011',
    'FEMMES SOLIDAIRES':'42265972200017',
    'VILLETTE MAKERZ BY WOMA':'82265962900015',
    'WOMEN ENGAGE FOR A COMMON FUTURE - FRANCE':'50457276900025',
    'LA COMEDIE ITALIENNE':'32486312500013',
    'MINI-COCCINELLES':'42088122900025',
    'LA  QUEUE DE LA COMETE':'47948319000027',
    'COLLECTIF PREMATURITE':'79433911900024',
    'ASSOCIATION DES COMMERÇANTS BELLEVILLOIS':'80169535400010',
    'ATELIER KUSO':'54003540900049',
    'COMPAGNIE DARIUS':'42099450100013',
    'COMPAGNIE NARCISSE THEATRE':'48912947800058',
    'LA CINÉMATHÈQUE FRANÇAISE':'77567146400110',
    'S.O.S. PARIS':'32239771200017',
    'COMITE CAMI ILE DE FRANCE':'43800912800044',
    'COMPAGNIE DES MORNES BLEUS':'44290536000024',
    'CIE DHANG-DHANG !':'50439334900023',
    'WOMEN AND HEALTH ALLIANCE INTERNATIONAL':'51331646300027',
    'LES ITEMS ASSOCIÉS':'48113998800036',
    'LE PONT QIAOLIANG':'44006098600039',
    'PARIS POTIER':'51531476300027',
    'ASP FONDATRICE':'33289091200048',
    'LE HUIT':'80427611100015',
    'ASSOCIATION SPORTIVE DU COLLEGE ELSA TRIOLET':'48122039000019',
    'LES AMIS DE L\'ENIJE PARIS/TOGO':'53820724200016',
    'FOOD SWEET FOOD':'83941050300016',
    'ACCÈS À LA LECTURE POUR TOUS':'49360931700018',
    'ASSOCIATION DES PERSONNELS SPORTIFS DES ADMINISTRATIONS PARISIENNES DE LA VILLE DE PARIS APSAP':'77566270300047',
    'ASSOCIATION CHIDA COMORES':'47890689400025',
    'ASSOCIATION POUR LA DÉFENSE ET L\'ANIMATION DES COMMERCES ET ENTREPRISES DU QUARTER MONTORGUEIL':'78931903500026',
    'ASSOCIATION FRANÇAISE DE PÉDIATRIE AMBULATOIRE':'40125591400103',
    'VACANCES & FAMILLES ILE-DE-FRANCE':'31081902400293',
    'SCIENCE ET SPORT':'45230442100038',
    'L\'ATELIER DE THEATRE VIVANT':'75389163900035',
    'MAM\'AYOKA, LA CUISINE DU MONDE À CÔTÉ':'81163393200041',
    'FEDERATION INTERNATIONALE MUSIQUE ESPERANCE':'44262410200038',
    'LETTRE AIMÉ':'81828382200018',
    'COMPAGNIE A BOUT PORTANT':'51840173200038',
    'ART EXPRESS-COMPAGNIE TATOO':'38476267000030',
    'PIXIFLORE':'44868553700020',
    'ASSOCIATION UNIVERSELLE SENIOR ENTREPRISE':'75201007400017',
    'PAIN O CHOKOLAT':'52073405400015',
    'PHOTOGRAPHES PARISIENS':'52191768200010',
    'COMMUNAUTE DU BOULKIEMDEEN FRANCE C.B.F.':'75193882000014',
    '59RIVOLI':'45386579200015',
    'ISATIS':'38324753300094',
    'ASSOCIATION NATIONALE DES DIRECTEURS ET DES INTERVENANTS D¿INSTALLATIONS ET DES SERVICES DES SPORTS':'49268203400052',
    'L\'ETRANGE FESTIVAL':'40816295600010',
    'PARIS MECONNUES':'50904139800017',
    'ENT\'REVUES':'35259204200050',
    'BABILLAGES':'32079827500030',
    'COMITÉ VENDÔME':'78417863400050',
    'ASSOCIATION DES COMMERÇANTS DU HAUT MONTMARTRE':'53203555700018',
    'GRANDE VIGIE':'51758557600021',
    'OBJECTIF50':'48766411200033',
    'R2SONGES':'43524242500039',
    'TRÈS D\'ESPRIT':'44298598200028',
    'PERSONA TRES GRATA':'53439381400015',
    'ASSOCIATION ARTISTIQUE COMPAGNIE ELIZABETH CZERCZUK':'82837810900015',
    'AVRON CONCEPT':'50977060800024',
    'UNION NATIONALE DES ANCIENS COMBATTANTS D\'INDOCHINE, DES TOE ET D\'AFRIQUE DU NORD':'49754112800029',
    'COOPCYCLE':'83361956200014',
    'EN VOTRE COMPAGNIE':'49473689500058',
    'L\'AMICALE DU CONSERVATOIRE DU 8 EME':'48021100200013',
    'COMPAGNIE LES GENS QUI TOMBENT (TOUS TORTS PARTAGÉS)':'53473451200047',
    'LES MOTS DES FEMMES D\'ICI ET D\'AILLEURS':'53163981300017',
    'LES UNS AVEC LES AUTRES':'79299483200014',
    'CARAVANE FILMS':'52976531500013',
    'ASSOCIATION DES COMMERÇANTS DE LA RUE DE VOUILLÉ':'83526701400024',
    'L\'INSTITUT MANGALA':'49157339000011',
    'SOLINUM':'82169115100020',
    'LA BELLE ENVOLÉE':'40924412600010',
    'FÉDÉRATION EUROPÉENNE DES BANQUES ALIMENTAIRES':'35072721000022',
    'MULVABE DANSE':'38036951200047',
    'LIEN POPI PERINATALITE ORIENTATION PSYCHANALYTIQUE ET INSTITUTION':'80962186500017',
    'EDITIONS ET PUBLICATIONS DE L\'ECOLE LACANIENNE':'38928358100038',
    'LE JOUR DE LA TERRE':'38005508700031',
    'LE THÉÂTRE DU PEUPLE LIÉ':'80347680300017',
    'ASSOCIATION SPORTIVE DU COLLÈGE PARIS NORD EST':'80919931800019',
    'VIENS, JE T\'EMMÈNE':'50119426000029',
    'EN AVANT TOUTE II':'53415664100035',
    'AU TOUR DU CANAL DE L\'OURCQ':'51272671200023',
    'PÂTISSERIES SOLIDAIRES':'82835674100011',
    'LA RENAISSANCE FRANÇAISE CULTURE - SOLIDARITÉ - FRANCOPHONIE':'79846019200015',
    'LES COMPAGNONS DU 8 NOVEMBRE 1942-ACTES DE RÉSISTANCE : MÉMOIRE ET RECHERCHE':'81779688100024',
    'COMITÉ INTERNATIONAL DE LA CROIX ROUGE':'30320079400038',
    'COMPAGNIE LES CHIENS DE PAILLE':'52977822700023',
    'ASSOCIATION DES HABITANTS DU 179 RUE DE CHARONNE':'82781535800011',
    'LE GRENIER DE BABOUCHKA':'48025789800019',
    'FÉDÉRATION ACCUEIL MARIANNE 18E':'51390957200010',
    'ASSOCIATION CULTURELLE ET CARITATIVE PASSY-MOZART':'52328855300010',
    'THEATRE DE L\'ERRE':'39088431000041',
    'LAÏKA':'82804545000024',
    'THÉÂTRE VOLLARD':'32635747200025',
    'AXE DE SOLIDARITE POUR LA PROMOTION A L\'EDUCATION POUR TOUS - ASPECT':'52513378100019',
    'INSTANCE RÉGIONALE D\'ÉDUCATION ET DE PROMOTION DE LA SANTÉ D\'ILE DE FRANCE':'52310581500014',
    'ASSOCIATION DE SOLIDARITE DES COMBATTANTS DE L\'INDEPENDANCE DU BANGLADESH EN FRANCE':'83931488700010',
    'ASSOCIATION POUR L¿HISTOIRE VIVANTE - CENTRE D¿ORGANISATION D¿EXPOSITIONS ANNUELLES D¿HISTOIRE EXPLIQUEE':'44817968900022',
    'LES MAILLONS DE L\'ESPOIR':'84412703500013',
    'MAD RHIZOME PRODUCTION':'79477724300021',
    'INDE ESPOIR':'42847164300018',
    'CHOEUR D\'ARTY SHOW':'82948322100018',
    'ASSOCIATION DES VICTIMES DE L\'EXPLOSION RUE DE TRÉVISE DU 12 JANVIER 2019':'85025515900012',
    'CORPO MATERIAL':'83461206100012',
    'CENTRE JEUNESSE PARIS 18E NORD':'38909785800010',
    'HORSCHAMP - RENCONTRES DE CINÉMA':'83364114500010',
    'FEDERATION FRANCAISE DE TIR A L\'ARC F.F.T.A.':'78475437600068',
    'MUSIQUE JEUNE PUBLIC':'39927583300024',
    'ASSOCIATION PARISIENNE D\'AIKIDO TRADITIONNEL':'42888977800018',
    'COLLECTIF MAQUIS\'ARTS':'48277066600030',
    'DOJO CLUB SPORTIF DU MARAIS':'39903961900016',
    'ESPOIR BERBÈRE':'83834612000010',
    'MEMOIRE ET PATRIMOINE DES TERRE-NEUVAS':'50186170200022',
    'LA COMPAGNIE DES EPICES':'39287205700055',
    'LES JARDINS DE LA MONTAGNE VERTE':'44068763000051',
    'ASSOCIATION DES SOURCES DE LA SEINE':'50097582600016',
    'AMITIÉS INTERNATIONALES ANDRÉ MALRAUX':'42304190400016',
    'SIPARKA':'81536509300013',
    'PARIS 14 TERRITOIRE DE CINEMA':'83479403400019',
    'FORMICA PRODUCTION':'85406421900017',
    'COMPAGNIE LES EXQUISES':'88459592700010',
    'REBUS':'34395590200030',
    'KALEÏDO SCOPE':'78946229800025',
    'LIVREURS D\'ESPOIR':'82873186900016',
    'LE HAUT PAVE GALERIE D\'ESSAI':'32363567200013',
    'ASSOCIATION FRANÇAISE DES VICTIMES DU TERRORISME (AFVT)':'51348142400044',
    'HALTE GARDERIE DE L\'ANNONCIATION':'31164133600012',
    'ROSE':'42843534100022',
    'ASSOCIATION D\'ENTRAIDE AUX PUPILLES ET ANCIENS PUPILLES DE L\'ÉTAT':'35097849000026',
    'GROUPE RIRES RAGE RÉSISTANCE':'42294808300031',
    'ASSOCIATION FRANCAISE DES VICTIMES DU TERRORISME AFVT':'51348142400044',
    'ASSOCIATION NATIONALE DE RECHERCHE ET D\'ACTION THEATRALE ANRAT THEATRE-EDUCATION':'34435422000040',
    'ASSOCIATION POUR LE SOUVENIR DE FUSILLÉS DU MONT VALÉRIEN ET DE L\'ILE DE FRANCE':'48250860300016',
    'COLLECTIF SAUF LE DIMANCHE':'49773587800025'
}

In [23]:
# Fill the DataFrame with correct SIRET or drop the line if no corrected SIRET

for nom in dct_siret_nom_incorrect.values():
    try:
        data.loc[data['Nom Bénéficiaire'] == nom, 'Numéro Siret'] = dct_nom_siret_incorrect[nom]
    except:
        df_drops = pd.concat([df_drops,data.loc[data['Nom Bénéficiaire'] == nom]], ignore_index=True)
        data.drop(data.loc[data['Nom Bénéficiaire'] == nom].index, axis = 0, inplace=True)

In [24]:
# API call for corrected SIRETs

df, rejects = get_data_from_sirets(dct_nom_siret_incorrect.values(), lst_fields)
if not df.empty:
    print(f'Appending to dataframe...')
    data_sirets = pd.concat([data_sirets,df], ignore_index=True)

Request OK
Found 170/170 SIRETs
Appending to dataframe...


In [40]:
# Last treatment: we will not be able to use SIRENE information
# for entities without 'fields.statutdiffusionunitelegale' = 'O'
# so we should drop them

try:
    count = 0
    for siret in data_sirets.loc[data_sirets['fields.statutdiffusionunitelegale'] != 'O', 'fields.siret']:
        df_drops = df_drops = pd.concat([df_drops,data.loc[data['Numéro Siret'] == siret]], ignore_index=True)
        data.drop(data.loc[data['Numéro Siret'] == siret].index, axis = 0, inplace=True)
        count +=1

    data_sirets.drop(data_sirets.loc[data_sirets['fields.statutdiffusionunitelegale'] != 'O'].index, axis=0, inplace = True)

    print (f'{count} record(s) dropped due to restricted status')

except:
    pass


1 record(s) dropped due to restricted status


### CHECKS

In [41]:
# Do we still have the same number of records?
print(df_drops.shape[0])
print(data.shape[0])
df_drops.shape[0] + data.shape[0] == nb_rows_original

111
79973


True

### SAVE TO CSV

In [42]:
df_drops.to_csv('../00_DataFiles/01_Cleaned/ParisSubsidies_DroppedRecords.csv')
data_sirets.to_csv('../00_DataFiles/01_Cleaned/SIRENE_EntitiesInformation.csv')
data.to_csv('../00_DataFiles/01_Cleaned/ParisSubsidies_Records.csv')